In [1]:
# import torch
import numpy as np
import scipy
import os

np.set_printoptions(precision=3, suppress=True, linewidth=10000, threshold=100000)

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# If the fonts in the plots are incorrectly rendered, comment out the next two lines
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
matplotlib.rcParams['lines.linewidth'] = 2.0

import urllib.request
from urllib.error import HTTPError

# CHECKPOINT_PATH = "../../saved_models/DL2/GDL"

/tmp/ipykernel_11716/1119636429.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [2]:
from escnn.group import *


/homes/ebm32/escnn_jax/escnn/group/groups/so3group.py:20: UserWarning: `py3nj` package not found! Will use a numerical method to compute the SO(3) Clebsh-Gordan coefficents. This is much slower but the coefficients will be cached on disk.
  warnings.warn("`py3nj` package not found! Will use a numerical method to compute the SO(3) Clebsh-Gordan coefficents. This is much slower but the coefficients will be cached on disk.")


In [3]:
G = cyclic_group(N=8)

# We can verify that the order of this group is 8:
G.order()

8

In [4]:
G.identity

0[2pi/8]

In [9]:
G.sample()


3[2pi/8]

In [10]:
a = G.sample()
b = G.sample()
print(a)
print(b)
print(a @ b)
print(~a)

2[2pi/8]
1[2pi/8]
3[2pi/8]
6[2pi/8]


In [12]:
rho = G.trivial_representation

In [13]:
g1 = G.sample()
g2 = G.sample()
print(rho(g1) @ rho(g2))
print(rho(g1 @ g2))

[[1.]]
[[1.]]


In [14]:
rho.size

1

In [15]:
G = so2_group()
rho = G.standard_representation()

g1 = G.sample()
g2 = G.sample()
print(f'g1={g1}, g2={g2}, g1 * g2 = {g1 @ g2}')
print()
print('rho(g1) @ rho(g2)')
print(rho(g1) @ rho(g2))
print()
print('rho(g1 * g2)')
print(rho(g1 @ g2))

g1=0.7103245605553715, g2=5.306383668324507, g1 * g2 = 6.016708228879879

rho(g1) @ rho(g2)
[[ 0.965  0.263]
 [-0.263  0.965]]

rho(g1 * g2)
[[ 0.965  0.263]
 [-0.263  0.965]]


In [17]:
rho_sum = rho + rho

g = G.sample()
print(rho(g))
print()
print(rho_sum(g))

[[-0.255  0.967]
 [-0.967 -0.255]]

[[-0.255  0.967  0.     0.   ]
 [-0.967 -0.255  0.     0.   ]
 [ 0.     0.    -0.255  0.967]
 [ 0.     0.    -0.967 -0.255]]


In [18]:
rho.size, rho_sum.size


(2, 4)

In [19]:
rho_sum = rho + rho + rho + rho

# or, more simply:
rho_sum = directsum([rho, rho, rho, rho])
rho_sum.size

8

In [24]:
G = cyclic_group(8)
rho = G.regular_representation

In [25]:
rho.size

8

In [26]:
rho(G.identity)

array([[ 1.,  0., -0.,  0., -0., -0.,  0., -0.],
       [ 0.,  1.,  0., -0., -0., -0., -0., -0.],
       [-0.,  0.,  1., -0., -0.,  0., -0.,  0.],
       [ 0., -0., -0.,  1.,  0., -0., -0., -0.],
       [-0., -0., -0.,  0.,  1.,  0., -0., -0.],
       [-0., -0.,  0., -0.,  0.,  1., -0.,  0.],
       [ 0., -0., -0., -0., -0., -0.,  1., -0.],
       [-0., -0.,  0., -0., -0.,  0., -0.,  1.]])

In [27]:
rho(G.element(1))


array([[ 0., -0.,  0., -0., -0.,  0., -0.,  1.],
       [ 1.,  0., -0., -0., -0., -0.,  0., -0.],
       [-0.,  1.,  0., -0.,  0., -0., -0., -0.],
       [-0.,  0.,  1.,  0., -0.,  0., -0.,  0.],
       [-0., -0., -0.,  1.,  0., -0.,  0., -0.],
       [-0.,  0., -0.,  0.,  1.,  0.,  0., -0.],
       [-0., -0., -0., -0.,  0.,  1.,  0.,  0.],
       [-0.,  0., -0., -0.,  0., -0.,  1.,  0.]])

In [28]:
f = np.zeros(8)
f[0] = 1
f

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
rho(G.identity) @ f


array([ 1.,  0., -0.,  0., -0., -0.,  0., -0.])

In [30]:
from escnn import group
from escnn import gspaces
from escnn import nn

In [31]:
r2_act = gspaces.rot2dOnR2(N=4)
r2_act

C4_on_R2[(None, 4)]

In [32]:
# we can access the group G as
G = r2_act.fibergroup
G

C4

In [33]:
feat_type = nn.FieldType(r2_act, [G.regular_representation])
feat_type

[C4_on_R2[(None, 4)]: {regular (x1)}(4)]

In [34]:
G.regular_representation(G.sample())


array([[-0., -0.,  1.,  0.],
       [ 0., -0., -0.,  1.],
       [ 1.,  0., -0., -0.],
       [ 0.,  1.,  0., -0.]])

In [35]:
feat_type = nn.FieldType(r2_act, [G.regular_representation]*3)
feat_type

[C4_on_R2[(None, 4)]: {regular (x3)}(12)]

In [36]:
feat_type_in = nn.FieldType(r2_act, [r2_act.trivial_repr])
feat_type_out = nn.FieldType(r2_act, [r2_act.regular_repr]*3)

In [37]:
conv = nn.R2Conv(feat_type_in, feat_type_out, kernel_size=3)


AttributeError: module 'escnn.nn' has no attribute 'R2Conv'